In [23]:
import pandas as pd
import gpxpy
import datetime
from geopy.distance import geodesic
import matplotlib.pyplot as plt

In [52]:
def parse_gpx_file(filepath):
    gpx_file = open(filepath, 'r')
    gpx = gpxpy.parse(gpx_file)

    data = []
    start_time = datetime.datetime(2024, 5, 27, 0, 0, 0)  # Start time
    
    for track in gpx.tracks:
        for segment in track.segments:
            for idx, point in enumerate(segment.points):
                data.append({
                    'time': point.time if point.time else start_time + datetime.timedelta(seconds=idx),
                    'latitude': point.latitude, 
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })

    df = pd.DataFrame(data)

    df['prev_latitude'] = df['latitude'].shift(1)
    df['prev_longitude'] = df['longitude'].shift(1)

    df = df.dropna().reset_index(drop=True)

    df['distance'] = df.apply(lambda x: geodesic((x['prev_latitude'], x['prev_longitude']), (x['latitude'], x['longitude'])).meters, axis=1)
    df['accumulated_distance'] = df['distance'].cumsum()
    
    return df

In [53]:
garmin_df = parse_gpx_file('../data/27-05-2023/garmin.gpx')
polar_df = parse_gpx_file('../data/27-05-2023/polar.gpx')
route_df = parse_gpx_file('../data/27-05-2023/route.gpx')

ValueError: Cannot set a DataFrame with multiple columns to the single column distance

In [ ]:
garmin_df.head()